## CAR construction


In [3]:
# Импорт библиотеку
%matplotlib inline
import pandas as pd
import deepdish as dd
import numpy as np
import os
import importlib
from tqdm import tqdm_notebook
import sys
sys.path.append('/home/dnazarenko/notebooks/RB/collections_rb_2/')
from CAR.processing import  credit
from CAR.processing import payments
from CAR import metadata
import gc
import datetime
from collections import defaultdict

In [4]:
# Вспомогательные константы 
# step and borders for CAR assembly
start = pd.to_datetime('2016-05-01')
end = pd.to_datetime('2018-11-01')
#test_date = pd.to_datetime('2019-02-24')
step= '1M'
debt_col = ['PTALNRP', 'PTPEN', 'PTPENI', 'PTSMPO', 'PTSMPP', 'PTUSR21', 'PTZAD']
metadata.out_dir = '/home/fshare_rb/rb/hdfs/'
car_dir = '/home/fshare_rb/rb/CAR_regression_new/'

In [5]:
# Загрузка данных платежей и кредитов
# Big dataframes are loaded one time and continuously used in CAR assembly
df_payments_cleaned = dd.io.load(metadata.out_dir+'payments_cleaned.h5')
print('loaded payments')
df_credit = dd.io.load('/home/fshare/rb/data_1712/hdfs/credit.h5')
print('loaded contact credit info')

loaded payments
loaded contact credit info


### NEW CREDIT PREPROCESSING START

In [6]:
# Добавление переменных в таблицу кредитов
df_credit['age']= (df_credit['ptdtn']-df_credit['ptbsd']).dt.days/365.25
df_credit['planned_duration'] = (df_credit['ptdtk']-df_credit['ptdtn']).dt.days/30
df_credit['id_cl'] = df_credit['SRC_STM_ID'] + '/' +df_credit['ac13'].str[5:11]

In [7]:
# Преобразование типов датафрейма (из строковых в числовые)
dummy_cols = ['HUB','CLIENT_CATEGORY_BUDGET','PRODUCT_ENG8N','cusex',]
num_cols = ['irat','ptsum',]
key_cols = ['ac13_orig','ptdtn']
for col in num_cols:
    df_credit[col] = pd.to_numeric(df_credit[col].str.replace(',','.'))

In [8]:
# Преобразование части строковых колонок в категориальные
credit_cols = dummy_cols+num_cols+key_cols
df_credit_clean = pd.get_dummies(df_credit[credit_cols],columns=dummy_cols)
# tmp.shape,df_credit.shape

### NEW CREDIT PREPROCESSING END

In [9]:
# Загрузка данных статуса ссуд и портфолио

df_status = dd.io.load('/home/fshare_rb/rb/hdfs/status_month.h5')
df_status.ptdtn = pd.to_datetime(df_status.ptdtn)

df_2016 = dd.io.load('/home/fshare_rb/rb/hdfs/portfolio_daily_2016.h5')
df_2017 = dd.io.load('/home/fshare_rb/rb/hdfs/portfolio_daily_2017.h5')
df_2018 = dd.io.load('/home/fshare_rb/rb/hdfs/portfolio_daily_2018.h5')
df_2019 = dd.io.load('/home/fshare_rb/rb/hdfs/portfolio_daily_2019.h5')

df_portfolio = pd.concat([df_2016,df_2017,df_2018,df_2019])
df_portfolio.sort_values(by = 'dt_beg',inplace=True)
df_portfolio = df_portfolio[~df_portfolio.delay_beg.isna()]
df_portfolio.loc[df_portfolio.dt_end.isna(),'dt_end'] = pd.to_datetime('2020-12-31')

del df_2016,df_2017,df_2018,df_2019
gc.collect()

21

In [10]:
# Ключи для соединений и фильтрации в ячейке ниже
keys = ['ac13_orig','ptdtn']
drop_cols = [
'PRODUCT_ENG8N',
'SRC_STM_ID',
'id_cl',
'car_credit_x',
'mortgage_x',
    'dt_beg',
    'dt_end', 
    'is_last',
]

In [11]:
# Функция добавления новых переменных к портфолио (число активных ссуд, наличие ипотеки и т.п.)
def add_features_to_pf(df_today_portfolio):
    debt_col = df_today_portfolio.columns[df_today_portfolio.columns.str.startswith('PT')]
    df_today_portfolio['debt'] = df_today_portfolio[debt_col].sum(axis=1)
    df_today_portfolio = df_today_portfolio[df_today_portfolio.debt>10000]
    df_today_portfolio = df_today_portfolio.merge(df_credit[keys+['PRODUCT_ENG8N','SRC_STM_ID']], 
                                                  on = keys,how='left')
    df_today_portfolio['id_cl'] = df_today_portfolio['SRC_STM_ID'] + '/' +df_today_portfolio['ac13'].str[5:11]
    active_credits = df_today_portfolio.groupby('id_cl').size().reset_index()
    active_credits.columns = ['id_cl','active_credits']
    df_today_portfolio = df_today_portfolio.merge(active_credits, on='id_cl',how='left')

    df_today_portfolio['car_credit'] = (df_today_portfolio.PRODUCT_ENG8N=='CAR-classic')| \
                                       (df_today_portfolio.PRODUCT_ENG8N=='CAR-express')
    df_today_portfolio['mortgage'] = df_today_portfolio.PRODUCT_ENG8N=='MORTGAGE'
    other_credits = df_today_portfolio.groupby('id_cl')[['car_credit','mortgage']].sum().reset_index()
    df_today_portfolio = df_today_portfolio.merge(other_credits, on='id_cl',how='left')
    df_today_portfolio.drop(drop_cols, axis=1,inplace=True)
    return df_today_portfolio

In [12]:
# Словарь с маппингом по дням просрочки
loan_keys = ['ac13_orig','ptdtn']
VAR_threshold = 4*10**5

dpd_mapping = {}
for i in range(5000):
    if i < 180:
        dpd_mapping[i] = '90-180'
    elif i < 360:
        dpd_mapping[i] = '180-360'
    elif i < 720:
        dpd_mapping[i] = '360-720'
    else:
        dpd_mapping[i] = '720+'
        

In [15]:
# Функция, собирающая CAR для отдельного месяца
fix_periods_num = 5
stats = {}
def get(date,
        df_credit_clean=df_credit_clean,
        df_payments_cleaned=df_payments_cleaned,
        due_days_list=(-180,-90,-15,-7,30,90),
        ):

    """
    Assemble CAR for a given date and save it to a file

    :param date
    :param df_contact_hist_preprocessed
    :param df_credit_clean
    :param df_payments_cleaned
    :param df_triggers
    :param due_days_list

    """
    date = pd.to_datetime(date)

    df_today_portfolio = df_portfolio[(df_portfolio.dt_beg<=date)&
                                      (df_portfolio.dt_end>=date)]
    print(df_today_portfolio.shape)
    df_today_portfolio = df_today_portfolio.merge(df_status[[str(date)[:10]]+loan_keys],on=loan_keys,how='inner')
    print('Status hard shape: {},after_merge_shape: {}'.format(df_status[~(df_status[str(date)[:10]].isna())].shape[0],
          df_today_portfolio.shape[0]))
        
    df_today_portfolio = add_features_to_pf(df_today_portfolio)
    print(df_today_portfolio.shape)
    df_today_portfolio = df_today_portfolio[~df_today_portfolio[str(date)[:10]].isna()]
    (print(df_today_portfolio.shape))
    print('Portfolio size', df_today_portfolio.shape)
    if df_today_portfolio.shape[0] < 1000:
        print('{}, empty portfolio, dropping'.format(date), fnm)
        return None
    
    df_today_portfolio = df_today_portfolio.merge(df_credit_clean,
                                                  how = 'inner',
                                                  on = loan_keys,
                                                 )
    df_today_portfolio['perc_left'] = df_today_portfolio['debt']/df_today_portfolio['ptsum']
    df= df_today_portfolio.drop_duplicates()
    
    
    
    for due_days in due_days_list:
        y_s = payments.get(date, df_payments_cleaned, due_days=due_days)
        y_s.ac13_orig = y_s.ac13_orig.astype(object)
        y_s.ptdtn = pd.to_datetime(y_s.ptdtn, errors = 'coerce')
        df = df.merge(y_s, on=['ac13_orig', 'ptdtn'], how='left')
        df.drop_duplicates(inplace=True)
    fix_paym_periods = payments.get(date, df_payments_cleaned, due_days=due_days,fix_periods=True)
    df = df.merge(fix_paym_periods,on=['ac13_orig','ptdtn'],how='left')
    
    #add variables
    df['date'] = date
    df['year'] = date.year
    df['month'] = date.month
    df['since_ptdtn_months'] = (df['date']-df['ptdtn']).dt.days/30
    df['dpd'] = (df['date']-df['delay_beg']).dt.days
    for i in range(1,4):
        df['cbr_minus_month_{}'.format(i)] = df['coll_total_minus_month_{}'.format(i)].fillna(0)/df['debt']*100
    df['quarter_cbr'] = sum([df['coll_total_minus_month_{}'.format(i)] for i in range(1,4)])/3
    dd.io.save('/home/fshare_rb/rb/CAR_regression_new/c_{}.h5'.format(str(date)[:10]), df)
    print('saved CAR part for {}, size: {}'.format(str(date)[:10],df.shape[0]))
    print('==============================================')
    return df

In [16]:
%%time
# Cборка CAR по месяцам
for time_point in tqdm_notebook(pd.date_range(start, end, freq=step)):
    time_point = time_point+pd.Timedelta(days=1)
    df = get(time_point)

(137141, 14)
Status hard shape: 135424,after_merge_shape: 108451
(107146, 16)
(97558, 16)
Portfolio size (97558, 16)


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:66: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['ac13_orig', 'ac13', '2016-06-01', 'coll_ov_-180', 'coll_ov_-90', 'coll_ov_-15', 'coll_ov_-7', 'coll_ov_30', 'coll_ov_90']]



saved CAR part for 2016-06-01, size: 88521
(138633, 14)
Status hard shape: 138485,after_merge_shape: 109769
(108446, 16)
(99104, 16)
Portfolio size (99104, 16)


KeyboardInterrupt: 

In [18]:
df.head().T

0                    1  \
ac13_orig                                9718522963201        9659Z14647200   
ac13                                     9718522963201        9659Z14647200   
ptdtn                              2012-01-13 00:00:00  2011-09-27 00:00:00   
PTZAD                                                0                    0   
PTSMPO                                         28436.2               739756   
PTSMPP                                               0               190349   
PTUSR21                                              0                    0   
PTALNRP                                         5698.6              75420.8   
PTPEN                                          73392.8               771056   
PTPENI                                          540.03               381335   
delay_beg                          2013-12-13 00:00:00  2013-03-27 00:00:00   
2016-06-01                                           x                    u   
debt                                            108068          2.15792e+06   
active_credits                                       2                    1   
car_credit_y                                         1                    1   
mortgage_y                                           0                    0   
irat                                              21.9                   19   
ptsum                                            75000               918913   
HUB_ДАЛЬНЕВОСТОЧНАЯ                                  0                    0   
HUB_МОСКОВСКАЯ                                       1                    1   
HUB_ПРИВОЛЖСКАЯ                                      0                    0   
HUB_СЕВЕРО-ЗАПАДНАЯ                                  0                    0   
HUB_СИБИРСКАЯ                                        0                    0   
HUB_ЮЖНАЯ                                            0                    0   
CLIENT_CATEGORY_BUDGET_Bona-fide                     1                    0   
CLIENT_CATEGORY_BUDGET_Depositors                    0                    0   
CLIENT_CATEGORY_BUDGET_Payroll                       0                    0   
CLIENT_CATEGORY_BUDGET_Walk-in                       0                    1   
PRODUCT_ENG8N_BIG money                              0                    0   
PRODUCT_ENG8N_CAR-classic                            0                    0   
...                                                ...                  ...   
coll_ov_-7_num                                     NaN                  NaN   
coll_ov_30                                         NaN                 True   
coll_ov_30_num                                     NaN                10163   
coll_ov_90                                         NaN                 True   
coll_ov_90_num                                     NaN              12771.2   
coll_ov_minus_month_1                              NaN                  NaN   
coll_total_minus_month_1                           NaN                  NaN   
coll_ov_minus_month_1_num                          NaN                  NaN   
coll_total_minus_month_1_num                       NaN                  NaN   
coll_ov_minus_month_2                              NaN                  NaN   
coll_total_minus_month_2                           NaN                  NaN   
coll_ov_minus_month_2_num                          NaN                  NaN   
coll_total_minus_month_2_num                       NaN                  NaN   
coll_ov_minus_month_3                              NaN                  NaN   
coll_total_minus_month_3                           NaN                  NaN   
coll_ov_minus_month_3_num                          NaN                  NaN   
coll_total_minus_month_3_num                       NaN                  NaN   
coll_ov_minus_month_4                              NaN                  NaN   
coll_total_minus_month_4                           NaN                  NaN   
coll_ov_minus_month_4_nu

## Combining Parts

In [38]:
# Замена boolean на int для таргет-колонок
def dtypes_coll_ov(df):
    for col in df.columns:
        if col.startswith('coll_ov') and not col.endswith('num'):
            print('re-typing:',col)
            df[col] = df[col].astype(np.float16)
    return df

In [39]:
%%time
# Сборка CAR из полученных выше частей
def combine_CAR(
             car_dir = car_dir,
            ):
    
    """
    loads and compiles car parts into one CAR, filters by delay horizon
    allows to select time boundaries and step for CAR loading
    
    :param delay_horizon
    :param step
    :param car_type
    :param car_dir
    """
    
    print('Loading ')
    car_parts_fnms = os.listdir(car_dir)
    
    
    car_parts = {}
   
    for fnm in car_parts_fnms[:]:
#         print(fnm)
        car_parts[fnm] = dd.io.load(car_dir+fnm)

    
    CAR = pd.concat(car_parts.values(),axis=0,sort=True)
    CAR = dtypes_coll_ov(CAR)
    print(CAR.shape)
    return CAR
    dd.io.save('/home/fshare_rb/rb/CAR/CAR_regression.h5',CAR)
    return CAR


CAR = combine_CAR()

In [43]:
# Загрузка истории взаоимодействий 
ch = pd.read_csv('/home/fshare/rb/data_2101/contact_history_2.txt',sep = "|", encoding='cp1251',error_bad_lines=False)

ch.rename(columns={'PTDTN':'ptdtn'},inplace=True)
ch.EXEC_DATE = pd.to_datetime(ch.EXEC_DATE,errors ='coerce')
ch.ptdtn = pd.to_datetime(ch.ptdtn,errors ='coerce')

b'Skipping line 114655: expected 18 fields, saw 19\nSkipping line 115492: expected 18 fields, saw 19\n'
b'Skipping line 190684: expected 18 fields, saw 19\n'
b'Skipping line 255301: expected 18 fields, saw 19\n'
b'Skipping line 941107: expected 18 fields, saw 19\nSkipping line 944405: expected 18 fields, saw 19\nSkipping line 945169: expected 18 fields, saw 21\n'
b'Skipping line 952450: expected 18 fields, saw 21\n'
b'Skipping line 1019983: expected 18 fields, saw 20\nSkipping line 1019996: expected 18 fields, saw 20\nSkipping line 1020156: expected 18 fields, saw 21\nSkipping line 1020170: expected 18 fields, saw 21\nSkipping line 1027313: expected 18 fields, saw 20\nSkipping line 1027319: expected 18 fields, saw 20\nSkipping line 1028109: expected 18 fields, saw 20\n'
b'Skipping line 1199667: expected 18 fields, saw 20\n'
b'Skipping line 1312673: expected 18 fields, saw 19\n'
b'Skipping line 1367996: expected 18 fields, saw 19\nSkipping line 1369787: expected 18 fields, saw 19\n'
b'S

In [44]:
#ID наиболее интересных действий в истории взаимодействий 
list_ID=[20402.0, 20542.0, 20372.0, 20148.0, 20619.0, 20670.0, 21077.0,
       21371.0, 20926.0, 20031.0, 20493.0, 21135.0, 20401.0, 21076.0,
       20644.0, 20438.0, 20398.0, 20769.0, 20770.0, 20761.0,
       21350.0, 20147.0, 20772.0, 20573.0, 20785.0, 20306.0, 20645.0,
       20399.0]

for ID in list_ID:
    CAR[str(ID)+'_time_dif'] = 0

In [46]:
# колонки в CAR, содержащие статус ссуды
stcols = CAR.columns[CAR.columns.str.startswith('201')]
stcols = stcols[:-2]
stcols

Index(['2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01',
       '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01',
       '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01',
       '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01',
       '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01',
       '2018-03-01', '2018-04-01', '2018-05-01', '2018-06-01', '2018-07-01',
       '2018-08-01', '2018-09-01', '2018-10-01', '2018-11-01'],
      dtype='object')

In [47]:
%%time
# Добавление в CAR счётчиков дней с момента события для каждого ID из списка выше.
for i,col in enumerate(tqdm_notebook(stcols)):
    for idshnik in list_ID:
#             print(idshnik)

        new_col = str(idshnik)+'_time_dif'
        proc = ch[ch["CHANNEL_ID"]==idshnik]
        proc = proc[proc.EXEC_DATE<col]
        proc = proc.groupby(['ac13_orig','ptdtn']).last().reset_index()
        proc[new_col] =(pd.to_datetime(col) - proc.EXEC_DATE).dt.days

        CAR.loc[(CAR.date==col),new_col] = CAR[(CAR.date==col)].merge(proc[['ac13_orig','ptdtn',new_col]], 
                                              on = ['ac13_orig','ptdtn'],
                                              how='left').fillna(0)[new_col+'_y'].values

CPU times: user 28min 29s, sys: 22.3 s, total: 28min 52s
Wall time: 28min 54s


In [55]:
# Маппинги регионов, VAR и дней просрочки для переменных-флагов по бакетам

# HUBS = ['HUB_ENGL_FAR-EAST', 'HUB_ENGL_MOSCOW', 'HUB_ENGL_NORTH-WEST',
#        'HUB_ENGL_SIBERIAN', 'HUB_ENGL_SOUTH', 'HUB_ENGL_VOLGA',
#        'HUB_ENGL_nan']

HUBS = ['HUB_ДАЛЬНЕВОСТОЧНАЯ', 'HUB_МОСКОВСКАЯ', 'HUB_ПРИВОЛЖСКАЯ',
       'HUB_СЕВЕРО-ЗАПАДНАЯ', 'HUB_СИБИРСКАЯ', 'HUB_ЮЖНАЯ']
dpd_mapping = {}
for i in range(5000):
    if i < 180:
        dpd_mapping[i] = '90-180'
    elif i < 360:
        dpd_mapping[i] = '180-360'
    elif i < 720:
        dpd_mapping[i] = '360-720'
    else:
        dpd_mapping[i] = '720+'

VAR_threshold = 4*10**5
CAR['VAR_bucket'] = (CAR['debt']>VAR_threshold).astype(int)
CAR['dpd_bucket'] = CAR.dpd.map(dpd_mapping)
CAR['HUB'] = CAR[HUBS].idxmax(axis=1)

In [58]:
# Добавление статистик по бакетам в переменные каждой ссуды

date_range = pd.date_range(start='2016-04-01',end='2018-11-01',freq='1M')
date_range = [date+pd.Timedelta(days=1) for date in date_range]

for var in ['VAR_bucket','dpd_bucket','HUB']:
    CAR[var+'_mean']=0
    CAR[var+'_std']=0
    CAR[var+'_skew']=0
    CAR[var+'_kurtosis'] = 0
    CAR[var+'_cbr']=0

def add_stats_to_CAR(CAR,window =3): 
    results = {}
    for i,col in enumerate(stcols):
        months = stcols[max(i-window,0):i]
        if len(months)<1:continue
        for var in ['VAR_bucket','dpd_bucket','HUB']:
            df = CAR[(CAR.date>=months[0])&
                           (CAR.date<=months[-1])]
            mean = df.groupby(var).coll_ov_30_num.mean().to_dict()
            std = df.groupby(var).coll_ov_30_num.std().to_dict()
            skew = df.groupby(var).coll_ov_30_num.skew().to_dict()
            kurtosis = df.groupby(var).coll_ov_30_num.apply(lambda x: x.kurtosis()).to_dict()
            df['cbr'] = df.coll_ov_30_num/df.debt*100
            cbr = df.groupby(var).cbr.mean()

            CAR.loc[(CAR.date==col),var+'_mean'] = CAR.loc[(CAR.date==col),var].map(mean)
            CAR.loc[(CAR.date==col),var+'_std'] = CAR.loc[(CAR.date==col),var].map(std)
            CAR.loc[(CAR.date==col),var+'_skew'] = CAR.loc[(CAR.date==col),var].map(skew)
            CAR.loc[(CAR.date==col),var+'_cbr'] = CAR.loc[(CAR.date==col),var].map(cbr)
    return CAR

In [59]:
# Сохранение типов CAR для каждого юнита (хард, хкц и т.п.)

car_types = {'hard':['x',],
             'odkp':['y',],
             'hcc':['z',],
             'external':['c','e','p'],
            }

results={}

for car_type, status_marker in tqdm_notebook(car_types.items()):
    print(car_type)
    CAR_parts = []
    for col in stcols:
        part = CAR[(CAR[col].isin(status_marker))&
                (CAR.date==col)]
        CAR_parts.append(part)

    results['CAR_{}'.format(car_type)] = pd.concat(CAR_parts,axis=0,ignore_index=True)
    print(results['CAR_{}'.format(car_type)].shape,CAR.shape)

    results['CAR_{}'.format(car_type)]= results['CAR_{}'.format(car_type)][results['CAR_{}'.format(car_type)].date<'2018-12-01']
    results['CAR_{}'.format(car_type)]= results['CAR_{}'.format(car_type)][results['CAR_{}'.format(car_type)].date>'2016-05-01']
    results['CAR_{}'.format(car_type)] = add_stats_to_CAR(results['CAR_{}'.format(car_type)])
    dd.io.save('/home/fshare_rb/rb/CAR/plans_new/CAR_{}.h5'.format(car_type),results['CAR_{}'.format(car_type)])

hcc
(274607, 160) (2569602, 160)


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:21: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block4_values] [items->['2016-02-01', '2016-03-01', '2016-04-01', '2016-05-01', '2016-06-01', '2016-07-01', '2016-08-01', '2016-09-01', '2016-10-01', '2016-11-01', '2016-12-01', '2017-01-01', '2017-02-01', '2017-03-01', '2017-04-01', '2017-05-01', '2017-06-01', '2017-07-01', '2017-08-01', '2017-09-01', '2017-10-01', '2017-11-01', '2017-12-01', '2018-01-01', '2018-02-01', '2018-03-01', '2018-04-01', '2018-05-

hard
(1325395, 160) (2569602, 160)
external
(559287, 160) (2569602, 160)
odkp
(96030, 160) (2569602, 160)
